In [1]:
import json
import requests


In [2]:
res = requests.get("https://randomuser.me/api/")
print(res.json())

{'results': [{'gender': 'female', 'name': {'title': 'Ms', 'first': 'Gertrude', 'last': 'Hunter'}, 'location': {'street': {'number': 4877, 'name': 'Eason Rd'}, 'city': 'Mildura', 'state': 'Western Australia', 'country': 'Australia', 'postcode': 6190, 'coordinates': {'latitude': '-87.5341', 'longitude': '19.4611'}, 'timezone': {'offset': '+9:00', 'description': 'Tokyo, Seoul, Osaka, Sapporo, Yakutsk'}}, 'email': 'gertrude.hunter@example.com', 'login': {'uuid': '57fe739f-df50-4604-bcb7-eb36834efc3e', 'username': 'lazytiger772', 'password': 'picture', 'salt': '9xaydrUY', 'md5': 'a409594fbc84f9cd7da6b0cc17bc4623', 'sha1': '0a4f4a89ceac21065d438158d3b8b547286d318f', 'sha256': 'b19ea3028aec4f3c5c68078931f9a896200ed186d27398ceae7b69037d6d4e1b'}, 'dob': {'date': '1956-07-14T15:10:26.988Z', 'age': 68}, 'registered': {'date': '2010-03-02T04:53:00.781Z', 'age': 15}, 'phone': '02-2997-2920', 'cell': '0481-561-101', 'id': {'name': 'TFN', 'value': '193768568'}, 'picture': {'large': 'https://randomuse